# Computer Vision Project

Submitted by: [*Mira Gleizer ID 320685118 and Dvora Daniel Ben-Tsvi ID 204186522*]

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

import winsound

## Part1
### In this part, we are going to modify the original video by adding the  at the end of half a minute of original video after being rotated. 
We rotated the video to test the function of saving the path.

In [2]:
def translate(dx,dy):
    M = np.eye(3)
    M[:,2] = [dx,dy,1]
    return M


def rotate(angle):
    M = np.eye(3)
    sin, cos = np.sin(angle), np.cos(angle)
    M[:2,:2] = [[cos,-sin],[sin,cos]]
    return M


#### The function for rotate the image

In [3]:
def RotateImage (frame, angle):
    h,w,_ = frame.shape
    
    T1 = translate(w/2, h/2)
    T2 = translate(-w/2, -h/2)

    angle =angle/100*2*np.pi
    M = rotate(angle)
    P = (T1.dot(M)).dot(T2)
    frame_warp = cv2.warpPerspective(frame.copy(), P, (w,h),borderValue=(255,255,255))

    return frame_warp

## The function  IfFrameBlue(frame, h) takes an image and determines if blue is the predominant color
### If blue is the predominant color - the function returns True, else returns False

In [4]:
def IfFrameBlue(frame, w, h):
   
    new_frame = cv2.resize(frame.copy(), (int(w/4), int(h/4)))
    #print (int(w/4), int(h/4))
    frame_hsv = cv2.cvtColor(new_frame, cv2.COLOR_BGR2HSV)
    bl = 0
    not_bl = 0

    for j in range (0, int(w/4)-1, 2):
        for i in range(0, int(h/4)-1, 2):
           
            temp = frame_hsv[i,j,:]
            if temp[0]<80 or temp[0]> 130:
                not_bl = not_bl+1

            else:
                bl = bl+1
   
    if not_bl>bl/2:
        return False
    else:
        return True


## The function writes blue frames to a new video file

In [5]:
def JustBlue(filenameIn, filenameOut):
    cap = cv2.VideoCapture(filenameIn)

    ret ,frame0 = cap.read()
    h,w,_ = frame0.shape # (height x width x 3)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # DIVX, XVID, MJPG, X264, WMV1, WMV2,...
    out = cv2.VideoWriter(filenameOut,fourcc, 20.0, (w,h)) #with border

    print(" Writing blue frames to a new video file. May be take a few minutes...")

    while True:
        ret, frame = cap.read()
        if ret==True:
            #cv2.imshow('Original',frame)
            if IfFrameBlue(frame,w,  h):
                out.write(frame)

        else:
            break
        
    cap.release() # release input video
    out.release() # release output video 
    print("Done");

## Here we "glue" the original part of the video together with edited part.
## Writing in beginning of the file - original partand adds in the end something like half a minute video
## from the rotated frames.

In [6]:
def Rotate_And_Glue(filenameIn1, filenameIn2, filenameOut):
    cap0 = cv2.VideoCapture(filenameIn1)
    cap = cv2.VideoCapture(filenameIn2)


    ret ,frame0 = cap0.read()
    h,w,_ = frame0.shape # (height x width x 3)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # DIVX, XVID, MJPG, X264, WMV1, WMV2,...
    out = cv2.VideoWriter(filenameOut, fourcc, 20.0, (w,h)) 

    print("we glue the original part of the video together with edited part. It may takes a few minutes....")

    # writing in beginning of the file - original part
    while True:
        ret, frame = cap0.read()
        if ret==True:

            out.write(frame)     
        else:
            break
    cap0.release() 

    ret, frame = cap.read()
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(int(n_frames/4)):
        ret, frame = cap.read()
        New_frame = RotateImage (frame,90)[50:310, 150:490]
        New_frame = cv2.resize(New_frame, (w, h))
        out.write(New_frame)

    for i in range(int(n_frames/4),int(n_frames/2)):
        ret, frame = cap.read()
        New_frame = RotateImage (frame,10)[50:310, 150:490]
        New_frame = cv2.resize(New_frame, (w, h))
        out.write(New_frame)

    cap.release() # release input video
    out.release() # release output video 

    cv2.waitKey(1)
    print("Done");

## Part 2
## In this part, we will process the video that we received in the previous part.
## First of all, we will determine the end of the pool track when the swimmer needs to stop or turn around. 
## Then, using the rotated part of the video, we will simulate the situation when the swimmer deviated from the straight route.
=======================================================================================================================

### The function DrawLines(frame) draws lines on the frame by using cv2.Canny() and  cv2.HoughLinesP(). 
Additionally the function calculates the slope of the line to find horizontal lines (The slope of the line is not more than 0.5 degrees)


In [15]:
def DrawLines(frame):
    flag = ""
    new_frame = frame.copy()
    gray_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
    edges_frame = cv2.Canny(gray_frame,100,200,apertureSize = 3) #detecting edges
    lines = cv2.HoughLinesP(edges_frame[100:, 150:450],1,np.pi/180,15,minLineLength=150,maxLineGap=7)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            #checking divine by zero
            if (x2-x1)==0:
                continue
            else:
                #calculating the slope of the line
                m = (y2 - y1)/(x2 - x1)      
               
                cv2.line(new_frame,(x1+150,y1+100),(x2+150,y2+100),(255, 100,0),2)
                if m > 0.5 and m < 2:
                    cv2.line(new_frame,(x1+150,y1+100),(x2+150,y2+100),(255, 255,255),2)
                    cv2.putText(new_frame,"<- Take left",(50,40), font, 0.5,(255,255,255),2,cv2.LINE_AA)
                    flag = "l"
                    
                if m > -2 and m < -0.5:
                    cv2.line(new_frame,(x1+150,y1+100),(x2+150,y2+100),(255, 255,255),2)
                    cv2.putText(new_frame,"Take right ->",(50,40), font, 0.5,(255,255,255),2,cv2.LINE_AA)
                    flag = "r"
                    
                if abs(m) < 0.5 :
                    cv2.line(new_frame,(x1+150,y1+100),(x2+150,y2+100),(0,0,255),2)

                    if IfLineIsRed(frame) :
                        cv2.putText(new_frame,"Red line",(50,40), font, 0.5,(255,255,255),2,cv2.LINE_AA)
                        continue
                    else:
                        cv2.putText(new_frame,"STOP!!!",(50,40), font, 1,(255,255,255),2,cv2.LINE_AA) 
                        cv2.putText(new_frame,"STOP!!!",(51,41), font, 1,(0,0,255),2,cv2.LINE_AA)
                        flag = "s"
                        

    if flag == "l" :
        winsound.Beep(1000, 50)
        winsound.Beep(1000, 100)
    if flag == "r" :
        winsound.Beep(1500, 100)
        winsound.Beep(1500, 50)
    if flag == "s" :
        winsound.Beep(2500, 50)
    return new_frame                  

### The Function IfLineIsRed(frame) checking if the detecting strip is red or blue, by using algorithm Kmeans for detecting centers of colors groups. 

In [16]:
def IfLineIsRed(frame):
    img_hsv = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2HSV)
    temp = []
    for i in range(0, h):
        temp.append([img_hsv[i,i,:][0], img_hsv[i,i,:][0]])
    temp = np.array(temp)
    temp = np.float32(temp) 
    
    centers_colors = use_kmeans(temp, 2)
   
    for c_c in centers_colors:
        
        if c_c[0] < 50 :
            return True
    return False

In [17]:
def use_kmeans(xy, k):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(xy, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    labels = labels.flatten()  # from shape (n,1) to (n,)

    return centers


### The Main

In [18]:
JustBlue('swimming_pool.mov', 'blue_out.mov')
Rotate_And_Glue('swimming_pool.mov', 'blue_out.mov', 'swimming_pool_with_rotate.mov')

cap = cv2.VideoCapture('swimming_pool_with_rotate.mov')

ret ,frame0 = cap.read()
h,w,_ = frame0.shape # (height x width x 3)

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # DIVX, XVID, MJPG, X264, WMV1, WMV2,...
out = cv2.VideoWriter('video_out.mov',fourcc, 20.0, (w,h)) 

font = cv2.FONT_HERSHEY_DUPLEX

while True:
    ret, frame = cap.read()
    if ret==True:
         
        cv2.imshow('Original',frame)
       
        if IfFrameBlue(frame,w,  h):
 
            New_frame = DrawLines(frame)
        else:
            New_frame = frame
        cv2.imshow('Project',New_frame)
        out.write(New_frame)
 
        if (cv2.waitKey(20) & 0xff == 27): # ESC key pressed?
            break

    else:
        break
        
cap.release() # release input video
out.release() # release output video 
cv2.destroyAllWindows() # delete output window
cv2.waitKey(1);

 Writing blue frames to a new video file. May be take a few minutes...
Done
we glue the original part of the video together with edited part. It may takes a few minutes....
Done
